In [ ]:
#creating State Summary Data

In [9]:
import pandas as pd
import numpy as np

In [7]:
#load and summarize the Wage and Cost Data
CostDF = pd.read_csv("CostDFwithPop.csv",index_col=0)
WageDF = pd.read_csv("WageDFwithPop.csv",index_col=0)

In [74]:
WageDF.where(WageDF['State'] == 'West Virginia').dropna(subset='State')

,WageType,1 ADULT - 0,1 ADULT - 1,1 ADULT - 2,1 ADULT - 3,2 ADULTS(1 WORKING) - 0,2 ADULTS(1 WORKING) - 1,2 ADULTS(1 WORKING) - 2,2 ADULTS(1 WORKING) - 3,2 ADULTS(BOTH WORKING) - 0,2 ADULTS(BOTH WORKING) - 1,2 ADULTS(BOTH WORKING) - 2,2 ADULTS(BOTH WORKING) - 3,State,County,2022 Population Est
8895,Living Wage,$15.38,$30.04,$38.33,$50.30,$24.31,$28.45,$34.15,$36.51,$12.15,$16.77,$22.00,$25.64,West Virginia,Barbour,15414.0
8896,Poverty Wage,$6.19,$8.38,$10.56,$12.74,$8.38,$10.56,$12.74,$14.92,$4.19,$5.28,$6.37,$7.46,West Virginia,Barbour,15414.0
8897,Minimum Wage,$8.75,$8.75,$8.75,$8.75,$8.75,$8.75,$8.75,$8.75,$8.75,$8.75,$8.75,$8.75,West Virginia,Barbour,15414.0
8898,Living Wage,$16.46,$33.17,$42.69,$56.75,$25.38,$30.52,$36.04,$39.26,$12.69,$18.34,$24.18,$28.86,West Virginia,Berkeley,129490.0
8899,Poverty Wage,$6.19,$8.38,$10.56,$12.74,$8.38,$10.56,$12.74,$14.92,$4.19,$5.28,$6.37,$7.46,West Virginia,Berkeley,129490.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9220,Poverty Wage,$6.19,$8.38,$10.56,$12.74,$8.38,$10.56,$12.74,$14.92,$4.19,$5.28,$6.37,$7.46,West Virginia,Wood,83340.0
9221,Minimum Wage,$8.75,$8.75,$8.75,$8.75,$8.75,$8.75,$8.75,$8.75,$8.75,$8.75,$8.75,$8.75,West Virginia,Wood,83340.0
9222,Living Wage,$15.26,$30.18,$38.47,$50.73,$24.14,$28.59,$34.29,$36.94,$12.07,$16.84,$22.07,$25.86,West Virginia,Wyoming,20527.0
9223,Poverty Wage,$6.19,$8.38,$10.56,$12.74,$8.38,$10.56,$12.74,$14.92,$4.19,$5.28,$6.37,$7.46,West Virginia,Wyoming,20527.0


In [11]:
StateNames = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 
              'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 
              'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
              'Maine', 'Maryland', 'Massachusetts','Michigan', 'Minnesota',
              'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 
              'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina',
              'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 
              'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 
              'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 
              'Wisconsin', 'Wyoming']

In [80]:
NewCostDF = pd.DataFrame(columns = CostDF.columns)
NewCostDF.drop(['County','2022 Population Est'],axis=1,inplace=True)
for state in StateNames:
    Dfstate = CostDF.where(CostDF['State'] == state).dropna(subset='State')
    StatePopSum = Dfstate.copy().drop_duplicates('County')
    SPSum = np.sum(StatePopSum['2022 Population Est'])
    #if state == 'West Virginia':
    #    print(SPSum)
    CountyList = set(list(Dfstate['County']))
    NewStateDF = pd.DataFrame(columns = Dfstate.columns)
    NewStateDF.drop(['State','County','2022 Population Est'],axis=1,inplace=True)
    for county in CountyList:
        countyweight = StatePopSum.where(StatePopSum['County'] == county).dropna(subset='County')['2022 Population Est']
        countyweight = float(countyweight/SPSum)
        #if state == 'West Virginia':
        #    print(countyweight)
        countyDF = Dfstate.where(Dfstate['County'] == county).dropna(subset='County')
        countyDF.drop(['State','County','2022 Population Est'],axis=1,inplace=True)
        toconcat = pd.DataFrame(columns = countyDF.columns)
        countyDF.set_index('CostType',inplace=True)
        for column in countyDF.columns:
            countyDF[column] = countyDF[column].apply(lambda x: str(x[1:]))
            countyDF[column] = countyDF[column].str.replace(',', '')
            countyDF[column] = countyDF[column].apply(lambda x: int(x)*countyweight)
        countyDF.reset_index(inplace=True) #was automatically named CostType
        toconcat = pd.concat([toconcat,countyDF])
        #print(toconcat)
        NewStateDF = pd.concat([NewStateDF,toconcat])
        #break
    NewStateDF = NewStateDF.groupby('CostType').sum()
    NewStateDF.reset_index(inplace=True)
    #noticed that middle group of 4 was missing the data that was the same in the left and right groups of 4:
    NewStateDF.at[1,'2 ADULTS(1 WORKING) - 1'] = NewStateDF.at[1,'1 ADULT - 1']
    NewStateDF.at[1,'2 ADULTS(1 WORKING) - 2'] = NewStateDF.at[1,'1 ADULT - 2']
    NewStateDF.at[1,'2 ADULTS(1 WORKING) - 3'] = NewStateDF.at[1,'1 ADULT - 3']
    NewStateDF['State'] = state
    #if state == 'West Virginia':
    #    print(NewStateDF)
    NewCostDF = pd.concat([NewCostDF,NewStateDF])

                              CostType   1 ADULT - 0    1 ADULT - 1  \
0                         Annual taxes  11877.821889   26450.575023   
1                           Child Care      0.000000   17431.986376   
2                                Civic   5006.000000    8038.000000   
3                                 Food   6702.000000    9874.000000   
4                              Housing  15314.288636   20055.377921   
5                              Medical   5198.000000   14414.000000   
6                                Other   9016.000000   13446.000000   
7   Required annual income after taxes  52500.434527  103057.336684   
8  Required annual income before taxes  64378.225915  129507.813007   
9                       Transportation  11018.000000   19552.000000   

     1 ADULT - 2    1 ADULT - 3  2 ADULTS(1 WORKING) - 0  \
0   33839.956707   44762.288431             20463.625263   
1   34864.877561   52296.575468                 0.000000   
2    9930.000000   12966.000000       

In [78]:
NewCostDF.where(NewCostDF['State'] == 'Virginia').dropna(subset='State')

,CostType,1 ADULT - 0,1 ADULT - 1,1 ADULT - 2,1 ADULT - 3,2 ADULTS(1 WORKING) - 0,2 ADULTS(1 WORKING) - 1,2 ADULTS(1 WORKING) - 2,2 ADULTS(1 WORKING) - 3,2 ADULTS(BOTH WORKING) - 0,2 ADULTS(BOTH WORKING) - 1,2 ADULTS(BOTH WORKING) - 2,2 ADULTS(BOTH WORKING) - 3,State
0,Annual taxes,7730.266463,14448.737188,18101.249131,23929.796124,11227.914916,13586.435689,15828.923631,17470.723886,11227.914916,15871.909508,20399.402207,24326.802827,Virginia
1,Child Care,0.0,9315.451248,18630.752635,27946.131736,0.0,9315.451248,18630.752635,27946.131736,0.0,9315.451248,18630.752635,27946.131736,Virginia
2,Civic,2503.0,4019.0,4965.0,6483.0,4019.0,4965.0,6483.0,5153.0,4019.0,4965.0,6483.0,5153.0,Virginia
3,Food,3351.0,4937.0,7417.0,9838.0,6143.0,7645.0,9856.0,11999.0,6143.0,7645.0,9856.0,11999.0,Virginia
4,Housing,12780.685592,15631.269908,15631.269908,20723.979224,13304.462297,15631.269908,15631.269908,20723.979224,13304.462297,15631.269908,15631.269908,20723.979224,Virginia
5,Medical,2736.0,8370.0,8157.0,8467.0,5679.0,8157.0,8467.0,8256.0,5679.0,8157.0,8467.0,8256.0,Virginia
6,Other,4508.0,6723.0,6150.0,8948.0,6723.0,6150.0,8948.0,8655.0,6723.0,6150.0,8948.0,8655.0,Virginia
7,Required annual income after taxes,31509.816348,58894.946772,73783.38912,97541.287723,45766.262927,55380.562299,64520.340105,71213.497508,45766.262927,64696.013547,83151.09274,99159.629244,Virginia
8,Required annual income before taxes,39240.109612,73343.431854,91884.854845,121471.034914,56994.183745,68966.979054,80348.984116,88684.056762,56994.183745,80567.779043,103550.499236,123486.368197,Virginia
9,Transportation,5509.0,9776.0,12709.0,15012.0,9776.0,12709.0,15012.0,16304.0,9776.0,12709.0,15012.0,16304.0,Virginia


In [69]:
NewWageDF = pd.DataFrame(columns = WageDF.columns)
NewWageDF.drop(['County','2022 Population Est'],axis=1,inplace=True)
for state in StateNames:
    Dfstate = WageDF.where(WageDF['State'] == state).dropna(subset='State')
    StatePopSum = Dfstate.copy().drop_duplicates('County')
    SPSum = np.sum(StatePopSum['2022 Population Est'])
    CountyList = set(list(Dfstate['County']))
    NewStateDF = pd.DataFrame(columns = Dfstate.columns)
    NewStateDF.drop(['State','County','2022 Population Est'],axis=1,inplace=True)
    for county in CountyList:
        countyweight = StatePopSum.where(StatePopSum['County'] == county).dropna(subset='County')['2022 Population Est']
        countyweight = float(countyweight/SPSum)
        countyDF = Dfstate.where(Dfstate['County'] == county).dropna(subset='County')
        countyDF.drop(['State','County','2022 Population Est'],axis=1,inplace=True)
        toconcat = pd.DataFrame(columns = countyDF.columns)
        countyDF.set_index('WageType',inplace=True)
        for column in countyDF.columns:
            countyDF[column] = countyDF[column].apply(lambda x: str(x[1:]))
            countyDF[column] = countyDF[column].str.replace(',', '')
            countyDF[column] = countyDF[column].apply(lambda x: float(x.strip())*countyweight)
        countyDF.reset_index(inplace=True)
        toconcat = pd.concat([toconcat,countyDF])
        NewStateDF = pd.concat([NewStateDF,toconcat])
        #break
    NewStateDF = NewStateDF.groupby('WageType').sum()
    NewStateDF.reset_index(inplace=True)
    NewStateDF['State'] = state
    NewWageDF = pd.concat([NewWageDF,NewStateDF])

In [75]:
NewWageDF.where(NewWageDF['State'] == 'West Virginia').dropna(subset='State')

,WageType,1 ADULT - 0,1 ADULT - 1,1 ADULT - 2,1 ADULT - 3,2 ADULTS(1 WORKING) - 0,2 ADULTS(1 WORKING) - 1,2 ADULTS(1 WORKING) - 2,2 ADULTS(1 WORKING) - 3,2 ADULTS(BOTH WORKING) - 0,2 ADULTS(BOTH WORKING) - 1,2 ADULTS(BOTH WORKING) - 2,2 ADULTS(BOTH WORKING) - 3,State
0,Living Wage,30.950441,62.263321,79.553352,105.235154,49.314067,58.632102,69.808941,75.582926,24.65569,34.647437,45.442212,53.595718,West Virginia
1,Minimum Wage,17.5,17.5,17.5,17.5,17.5,17.5,17.5,17.5,17.5,17.5,17.5,17.5,West Virginia
2,Poverty Wage,12.38,16.76,21.12,25.48,16.76,21.12,25.48,29.84,8.38,10.56,12.74,14.92,West Virginia


In [72]:
NewWageDF.to_csv('StateWageDF.csv')
NewCostDF.to_csv('StateCostDF.csv')